In [1]:
!pip3 install tqdm

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# import

In [2]:
import os
import math
import time
import numpy as np
from glob import glob
from tqdm import tqdm
from PIL import Image
from multiprocessing import Process, Queue

# Load

In [3]:
train_path = glob("./dataset/original/train/*.bytes")
test_path = glob("./dataset/original/test/*.bytes")
train_path[:5], test_path[:5]

(['./dataset/original/train/HDembPkLujCsic0oYQAp.bytes',
  './dataset/original/train/exC0jIpaqfdWOPwgrKoV.bytes',
  './dataset/original/train/idTa1lDvrZk38BJOLV96.bytes',
  './dataset/original/train/8j2uhZyFl6XGkCoixABY.bytes',
  './dataset/original/train/BoYuGUEWrgiReVDLPAbq.bytes'],
 ['./dataset/original/test/VlnP9seAEMy7Yit8SWv3.bytes',
  './dataset/original/test/Ks4xrBOT1nJPdXjMRqg2.bytes',
  './dataset/original/test/mYgoCUSKcfLRwMrOlz3a.bytes',
  './dataset/original/test/Ou1A5Cs6hSjYJEravXUf.bytes',
  './dataset/original/test/wxmsY5uEOS6vVXLk4tza.bytes'])

# def func

In [4]:
def convert_mlw_to_img(bytes_filepath, filename, width=28, height=28):
    with open(bytes_filepath) as hex_file:
        #Extract hex values
        hex_array = []
        for line in hex_file.readlines():
            hex_values = line.split()
            if len(hex_values) != 17:
                continue
            hex_array.append([int(i, 16) if i != '??' else 0 for i in hex_values[1:]])
        hex_array = np.array(hex_array)

    #Convert to an image of a specific size
    if hex_array.shape[1] != 16:
        assert (False)
    b = int((hex_array.shape[0] * 16) ** (0.5))
    b = 2 ** (int(math.log(b) / math.log(2)) + 1)
    a = int(hex_array.shape[0] * 16 / b)
    hex_array = hex_array[:a * b // 16, :]
    im = np.reshape(hex_array, (a, b))

    if width is not None and height is not None:
        #Downsample
        img = Image.fromarray(np.uint8(im))
        img = img.resize((width, height), resample=Image.Resampling.BICUBIC)
        img.save(filename)

# convert

In [ ]:
train_png_path = "./dataset/img/img128"

start = time.time()

processes = []
for path in tqdm(train_path):
    basename = os.path.basename(path).split(".")[0]
    p = Process(target=convert_mlw_to_img, args=(path, f"{train_png_path}/{basename}.png", 128, 128, ))
    p.start()
    processes.append(p)
    
for p in processes:
    p.join()
    
end = time.time()

print("수행시간: %f 초" % (end - start))

 97%|█████████▋| 10489/10868 [06:22<00:23, 16.17it/s]